# ML Modeling — Predicting Page Load Time

- Predicts `page_load_time_ms` using radio, device, operator, and context features.
- Time-aware train/test split (80/20%).
- Pipeline: scales numerics, one-hot encodes categoricals.
- Models: Linear, Decision Tree, Random Forest, Gradient Boosting.
- Evaluates R², RMSE, MAE; visualizes actual vs. predicted.
- Picks best by RMSE and saves the model.
- Surfaces feature importance for transparency.


In [ ]:
# Step 1: Imports & load
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from sklearn.utils import Bunch
from sklearn.inspection import permutation_importance

if os.path.exists("synthetic_qoe_sessions_clean.parquet"):
    df = pd.read_parquet("synthetic_qoe_sessions_clean.parquet")
else:
    df = pd.read_csv("synthetic_qoe_sessions_clean.csv", parse_dates=['timestamp'])

print(f"Loaded cleaned dataset with shape: {df.shape}")
display(df.head())


In [ ]:
# Step 2: Feature selection & preprocessing (predict page_load_time from radio+network)
TARGET = 'page_load_time_ms'

cat_features = ['device', 'operator', 'band']
num_features = ['rsrp_dbm', 'rsrq_db', 'sinr_db', 'app_size_kb', 'rtt_ms']

# Time context
df['hour'] = df['timestamp'].dt.hour
num_features = num_features + ['hour']

# Sort by time and build time-aware split (first 80% train, last 20% test)
df_sorted = df.sort_values('timestamp').reset_index(drop=True)

X = df_sorted[cat_features + num_features]
y = df_sorted[TARGET]

split_idx = int(0.8 * len(df_sorted))
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

# Preprocess: numeric median impute; categorical most_frequent + OHE
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
num_transformer = SimpleImputer(strategy='median')

preprocess = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_features),
        ('cat', cat_transformer, cat_features),
    ],
    remainder='drop'
)

# ---- Print remarks for interpretation ----
print("\n[Feature Selection & Preprocessing — PLT model]")

# Explicit checks to avoid leakage / redundant features
assert 'country' not in cat_features, "Do not include 'country' — negligible signal."
assert ('band' in cat_features) and ('network_type' not in cat_features), \
       "Use 'band' OR 'network_type' — not both. Chose 'band'."

print(f"- Target: {TARGET}")
print(f"- Categorical features: {cat_features}  (excluded: 'country'; chose 'band', not 'network_type')")
print(f"- Numeric features: {num_features}")

# Time-aware split summary
ts_start = df_sorted['timestamp'].iloc[0]
ts_split = df_sorted['timestamp'].iloc[split_idx-1]
ts_end   = df_sorted['timestamp'].iloc[-1]
print("- Split: time-aware (first 80% → train, last 20% → test) to avoid temporal leakage.")
print(f"  · Train window: {ts_start:%Y-%m-%d %H:%M} → {ts_split:%Y-%m-%d %H:%M}  (n={len(X_train):,})")
print(f"  · Test  window: {df_sorted['timestamp'].iloc[split_idx]:%Y-%m-%d %H:%M} → {ts_end:%Y-%m-%d %H:%M}  (n={len(X_test):,})")

# Preprocessing summary
print("- Preprocess:")
print("  · Numeric: SimpleImputer(strategy='median') → replaces missing values with median per column.")
print("  · Categorical: SimpleImputer(strategy='most_frequent') → replaces missing values with the most common category.")
print("  · OneHotEncoder(handle_unknown='ignore') → expands categorical variables into binary indicators.")
print("  · ColumnTransformer(remainder='drop') → only selected features pass through.")


In [ ]:
# Step 3: Train & evaluate multiple models via pipelines
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
}

fitted = {}
rows = []
for name, est in models.items():
    pipe = Pipeline(steps=[('preprocess', preprocess), ('model', est)])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)

    r2   = r2_score(y_test, y_pred)
    mse  = mean_squared_error(y_test, y_pred)
    rmse = float(np.sqrt(mse))
    mae  = mean_absolute_error(y_test, y_pred)

    rows.append({"Model": name, "R2": r2, "RMSE": rmse, "MAE": mae})
    fitted[name] = {"pipe": pipe, "y_pred": y_pred}

summary_df = pd.DataFrame(rows).sort_values("RMSE")
print("\n--- Model Performance Summary (time-aware split) ---")
display(summary_df)


In [ ]:
# Step 4: Pick best model (by RMSE), persist artefacts
best_row = summary_df.iloc[0]
best_name = best_row["Model"]
best_pipe = fitted[best_name]["pipe"]
print(f"Best model: {best_name}  |  RMSE={best_row['RMSE']:.4f}  R2={best_row['R2']:.3f}")

os.makedirs("artefacts", exist_ok=True)
os.makedirs("models", exist_ok=True)
summary_df.to_csv("artefacts/model_metrics.csv", index=False)

model_path = f"models/qoe_{TARGET}_best_model.joblib"
joblib.dump(best_pipe, model_path)
print(f"Saved best model → {model_path}")

# Mini model card
card = f"""# Model Card — QoE {TARGET}

**Best model:** {best_name}
**Train/Test split:** time-ordered, first 80% train / last 20% test
**Metrics (test):** R2={best_row['R2']:.3f}, RMSE={best_row['RMSE']:.4f}, MAE={best_row['MAE']:.4f}

**Features**
- Numerical: {', '.join(num_features)}
- Categorical (OHE): {', '.join(cat_features)}

**Notes**
- Radio KPIs (RSRP/RSRQ/SINR) and operator/band materially explain {TARGET}.
- Time-aware split avoids leakage from random shuffling on temporal data.
"""
with open("models/model_card.md", "w", encoding="utf-8") as f:
    f.write(card)
print("Wrote models/model_card.md")


In [ ]:
# Step 5: Feature importance (permutation) — top 15 on test split
from sklearn.inspection import permutation_importance
from typing import Any

# Permutation importance permutes original input columns (pre-pipeline),
# so the names are our original features:
model_features = cat_features + num_features

pi_result = permutation_importance(
    best_pipe,              # Pipeline
    X_test,                 # original feature DataFrame
    y_test,
    n_repeats=5,
    random_state=0,
    n_jobs=-1,
    scoring='neg_mean_squared_error'
)

def _pi_get(obj: Any, key: str):
    return getattr(obj, key, obj[key])

imp_mean = _pi_get(pi_result, "importances_mean")
imp_std  = _pi_get(pi_result, "importances_std")

pi_mean = pd.Series(imp_mean, index=model_features)
pi_std  = pd.Series(imp_std,  index=model_features)

imp = (
    pi_mean.sort_values(ascending=False)
           .rename("mean_importance")
           .to_frame()
           .assign(std=pi_std)
)

top = imp.head(15)

# ---- Print remarks for interpretation ----
print("\n[Permutation Importance — Explanation & Interpretation]")
print("- Method: Each feature is randomly permuted (shuffled) in X_test while keeping all others fixed.")
print("  · If the model’s MSE rises significantly → the feature was important.")
print("  · If the MSE barely changes → the feature had little predictive value.")
print("- Why: Unlike model coefficients, this directly measures feature contribution to prediction accuracy.")

fig, ax = plt.subplots(figsize=(12,4))
top.sort_values("mean_importance").plot.barh(y="mean_importance", xerr="std", legend=False, ax=ax)
ax.set_title(f"Permutation Importance (top 15) — {best_name}")
ax.set_xlabel("Importance (↑ = larger error when permuted)")
plt.tight_layout(); plt.show()

# ---- Print remarks for interpretation ----
print("- Interpretation of bar chart:")
print("  · Features at the top: highest impact — shuffling them causes the largest rise in MSE.")
print("  · Error bars (std): variation across 5 repeats; wider bars mean less stable importance.")
print("- Practical use: Focus optimization on features with high permutation importance; low-importance features can be candidates for dropping.")

# Save importances
os.makedirs("artefacts", exist_ok=True)
imp.to_csv(f"artefacts/permutation_importance_{TARGET}.csv")


In [ ]:
# Step 6: Diagonal plots — ALL models
import math, os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

os.makedirs("artefacts", exist_ok=True)

def _rmse(y_true, y_pred):
    try:
        return mean_squared_error(y_true, y_pred, squared=False)
    except TypeError:
        return math.sqrt(mean_squared_error(y_true, y_pred))

def diag_grid_all_models(y_true, fitted_dict, summary_df, target_name):
    # Order plots by RMSE (best first)
    model_order = summary_df.sort_values("RMSE")["Model"].tolist()
    n = len(model_order)
    cols = 2 if n > 1 else 1
    rows = math.ceil(n / cols)

    # ---- Print remarks for interpretation ----
    print("\n[Model Prediction vs Actual Scatter — Interpretation]")
    print("- Each subplot shows predicted vs actual Page Load Time (PLT) for one model.")
    print("- The red dashed line = ideal diagonal (perfect prediction).")
    print("- Points hugging the diagonal → model predictions are close to reality.")
    print("- Wider scatter from the diagonal → larger prediction error.")
    print("- RMSE in titles gives the average magnitude of these errors (lower = better).")

    # Common axis limits across all models
    y_min = float(min([np.min(y_true)] + [np.min(fitted_dict[m]["y_pred"]) for m in model_order]))
    y_max = float(max([np.max(y_true)] + [np.max(fitted_dict[m]["y_pred"]) for m in model_order]))

    # Shared axes to keep identical scales
    fig, axes = plt.subplots(rows, cols, figsize=(5*cols, 5*rows),
                             sharex=True, sharey=True, squeeze=False)

    last_idx = -1
    for i, m in enumerate(model_order):
        ax = axes[i // cols, i % cols]
        y_pred = fitted_dict[m]["y_pred"]
        rmse = _rmse(y_true, y_pred)
        ax.scatter(y_true, y_pred, alpha=0.25, s=12)
        ax.plot([y_min, y_max], [y_min, y_max], 'r--', lw=2)
        ax.set_title(f"{m}   RMSE={rmse:.0f}")
        ax.set_xlim([y_min, y_max])
        ax.set_ylim([y_min, y_max])
        ax.label_outer()  # hide inner tick labels
        last_idx = i

    # Remove unused axes (if any)
    for j in range(last_idx + 1, rows * cols):
        fig.delaxes(axes[j // cols, j % cols])

    # One set of common labels
    fig.text(0.5, 0.04, f"Actual {target_name}", ha='center')
    fig.text(0.04, 0.5, f"Predicted {target_name}", va='center', rotation='vertical')

    plt.tight_layout(rect=(0.06, 0.06, 1, 1))
    fig.savefig("artefacts/actual_vs_pred_all_models.png", dpi=150, bbox_inches="tight")
    plt.show()

# Call it
diag_grid_all_models(y_test, fitted, summary_df, TARGET)
